# Model 1: Train and Evaluation Fully Connected Neural Network

In [ ]:
from pathlib import Path
import torch 

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
import sys; sys.path.insert(0, '../')

In [ ]:
from models.fully_connected import FullyConnected, feature_columns, label_columns, get_loss_function, get_optimizer_function, train

## Read Datasets from .csv

In [ ]:
from utils.file_io import read_angle_datasets

In [ ]:
data_folder = Path("../../data/")
train_data, test_data = read_angle_datasets(data_folder, 0.9, feature_columns, label_columns)

In [ ]:
input_shape, output_shape = train_data[0][0].shape[0], train_data[0][1].shape[0]
print(f"Data shape {input_shape} / {output_shape} of total {len(train_data) + len(test_data)} data rows!")

## Load parameter, functions and dataloader

In [ ]:
from utils.file_io import define_dataloader_from_angle_dataset
import os
import ast
from dotenv import load_dotenv

In [ ]:
model_path = Path("../../models/fully_connected/").absolute()

In [ ]:
dotenv_path = model_path / ".env"
load_dotenv(dotenv_path=dotenv_path)

learning_rate = float(os.getenv("LEARNING_RATE"))
batch_size = int(os.getenv("BATCH_SIZE"))
hidden_activation = os.getenv("HIDDEN_ACTIVATION")
hidden_layers = ast.literal_eval(os.getenv("HIDDEN_LAYERS"))
dropout = float(os.getenv("DROPOUT"))
num_epochs = int(os.getenv("NUM_EPOCHS"))

In [ ]:
train_dataloader, validation_dataloader, test_dataloader = define_dataloader_from_angle_dataset(train_data, test_data, batch_size, split_size=0.95)

## Load and Train the model

In [ ]:
model = FullyConnected(input_shape, hidden_layers, output_shape, dropout, hidden_activation)

In [ ]:
_ = train(num_epochs, train_dataloader, validation_dataloader, model, get_loss_function(), 
                   get_optimizer_function(model, learning_rate), model_path, device)

## Evaluation

In [ ]:
from utils.evaluation import compute_losses_from, compute_predictions
from utils.visualization import create_trace_animation
from matplotlib import pyplot as plt
from IPython.display import HTML

In [ ]:
model_state_dict = torch.load(model_path / "checkpoint.pt")
model.load_state_dict(model_state_dict)

In [ ]:
model.eval()

## 1. Analysis: Compute test loss

In [ ]:
y, y_true = compute_predictions(test_dataloader, model, 'cpu')
test_losses = compute_losses_from(y, y_true, get_loss_function())
print(f"The mean squared error of the loaded model on test is: {test_losses.mean()}")

## 2. Analysis: Trace animation

In [ ]:
%matplotlib notebook
 
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

In [ ]:
animation = create_trace_animation(y.numpy(), y_true.numpy())
HTML(animation.to_jshtml())